# Importações

In [1]:
import numpy as np
from math import sqrt
from math import pi as PI
from math import atan
from math import sin
from math import cos

# Variáveis

In [2]:
a,b,c,d,e,f = [3,6,2,9,4,9]
tolerancia = 0.0000000000001

In [3]:
A = [[30 + a + f,          a,          b,      c,     d],
     [         a, 10 + b + e,          e,      f, a + b],
     [         b,          e, 50 + c + d,  b + c, c + d]]

x = [1,1,1,1,1]

# Operações Básicas

In [4]:
def matrixByMatrix(a,b):
    na = len(a)
    ma = len(a[0])
    nb = len(b)
    mb = len(b[0])
    
    mid = min(ma, nb)
    
    if ma != nb:
        raise Exception('Matrix Error!')
    
    c = [[0 for j in range(mb)] for i in range(na)]
    
    for i in range(na):
        for j in range(mb):
            for k in range(mid):
                c[i][j] += a[i][k] * b[k][j]
    
    return c

In [5]:
def matrixLHNorm2(a):
    n = len(a)
    m = len(a[0])
    soma = 0.0
    for j in range(m-1):
        for i in range(j+1,n):
            soma += a[i][j] * a[i][j]
    return soma

In [6]:
def Identidade(n,m):
    I = [[0 for i in range(m)] for j in range(n)]
    for i in range(min(n,m)):
        I[i][i] = 1
    return I

In [7]:
def Transposta(A):
    n = len(A)
    m = len(A[0])
    
    B = [[0 for j in range(n)] for i in range(m)]
    for i in range(n):
        for j in range(m):
            B[j][i] = A[i][j]
    return B

In [8]:
def printMatrix(A):
    for i in A:
        for j in i:
            s = "%.5f" % j
            print("%10s" % s, end = " ")
        print()

In [9]:
def decomposicaoLU(A):
    n = len(A)
    L = [[0 for j in range(n)] for i in range(n)]
    U = [[A[i][j] for j in range(n)] for i in range(n)]
    for i in range(n):
        L[i][i] = 1
    for j in range(n-1):
        for i in range(j+1, n):
            L[i][j] = U[i][j]/U[j][j]
            for k in range(j+1, n):
                U[i][k] -= L[i][j] * U[j][k]
            U[i][j] = 0
    return [L, U]

In [10]:
# L(Ux) = b; Ux = y
def solverLU(A, b):
    n = len(A)
    L, U = decomposicaoLU(A)
    
    # Finding y for Ly = b
    y = [i for i in b]
    for i in range(n):
        for j in range(0,i):
            y[i] -= L[i][j] * y[j]
    
    # Finding x for Ux = y
    x = [i for i in y]
    for i in range(n-1,-1,-1):
        for j in range(n-1,i,-1):
            x[i] -= U[i][j] * x[j]
        x[i] = x[i]/U[i][i]
    
    return x

# Método Jacobi

In [11]:
def metodoJacobi(A, e):
    n = len(A)
    m = len(A[0])
    tol = 0.000000000001
    
    var_norm = e + 1
    
    J = Identidade(n,m)
    A_new = [[j for j in i] for i in A]
    
    while var_norm > e:
        for j in range(0,n-1):
            for i in range(j+1,n):
                if(abs(A_new[i][j]) >= tol):
                    
                    if(abs(A_new[i][i] - A_new[j][j]) < tol):
                        ang = PI/4
                    else:
                        ang = atan(2.0*A_new[i][j]/(A_new[j][j] - A_new[i][i]))/2.0
                    
                    P = Identidade(n,m)
                    P[i][i] =  cos(ang)
                    P[j][j] =  cos(ang)
                    P[i][j] = -sin(ang)
                    P[j][i] =  sin(ang)
                    
                    Pt = Transposta(P)
                    
                    J = matrixByMatrix(J,Pt)
                    
                    A_new = matrixByMatrix(P, A_new)
                    A_new = matrixByMatrix(A_new, Pt)

        var_norm = matrixLHNorm2(A_new)

    return [A_new, J]

# Método QR

In [12]:
def decomposicaoQR(A):
    tol = 0.000000000001

    n = len(A)
    m = len(A[0])
    Q = Identidade(m,n)
    R = [[i for i in j] for j in A]
    
    for j in range(0,n-1):
        for i in range(j+1,n):
            if(abs(R[i][j]) >= tol):
                
                if(abs(R[j][j]) < tol):
                    ang = PI/2
                else:
                    ang = atan(R[i][j]/R[j][j])

                P = Identidade(n,m)
                P[i][i] =  cos(ang)
                P[j][j] =  cos(ang)
                P[i][j] =  sin(ang)
                P[j][i] = -sin(ang)

                Pt = Transposta(P)

                Q = matrixByMatrix(Q,P)
                R = matrixByMatrix(Pt,R)
    
    return [Q,R]

In [13]:
def metodoQR(A, e):
    n = len(A)
    m = len(A[0])
    
    P = Identidade(n,m)
    A_new = [[i for i in j] for j in A]
    var_norm = e + 1
    
    while var_norm >= e:
        Q, R = decomposicaoQR(A_new)
        A_new = matrixByMatrix(R,Q)
        P = matrixByMatrix(P,Q)
        
        var_norm = matrixLHNorm2(A_new)
    
    return [A_new, P]

# Achando a Matriz "U" (A.At com Jacobi)

In [14]:
AU = matrixByMatrix(A,Transposta(A))
auto_val, auto_vec = metodoQR(AU, tolerancia)
print("Auto valores:")
printMatrix(auto_val)
print("Auto vetores:")
printMatrix(auto_vec)

Auto valores:
4290.80768    0.00000    0.00000 
   0.00000 1668.45437    0.00000 
   0.00000    0.00000  479.73796 
Auto vetores:
   0.31146   -0.93741    0.15576 
   0.15582   -0.11132   -0.98149 
   0.93740    0.32997    0.11140 


In [15]:
n = len(auto_val)
vec_auto = []

for j in range(n):
    val = auto_val[j][j]
    vec = []
    for i in range(n):
        vec.append(auto_vec[i][j])
    vec_auto.append((val, vec))
vec_auto.sort(reverse=True)

U = [[0 for j in range(n)] for i in range(n)]
for i in range(n):
    for j in range(n):
        U[j][i] = vec_auto[i][1][j]
for i in range(n):
    print(vec_auto[i][0],end=" ")
print()
print("Matriz U:")
printMatrix(U)

4290.807677050978 1668.454365915156 479.73795703386537 
Matriz U:
   0.31146   -0.93741    0.15576 
   0.15582   -0.11132   -0.98149 
   0.93740    0.32997    0.11140 


# Achando a Matriz "V" (At.A com QR)

In [16]:
AV = matrixByMatrix(Transposta(A),A)
auto_val, auto_vec = metodoQR(AV, tolerancia)
print("Auto valores:")
printMatrix(auto_val)
print("Auto vetores:")
printMatrix(auto_vec)

Auto valores:
4290.80768    0.00000    0.00000    0.00000    0.00000 
   0.00000 1668.45437    0.00000    0.00000    0.00000 
   0.00000    0.00000  479.73796    0.00000    0.00000 
   0.00000    0.00000   -0.00000    0.00000   -0.00000 
  -0.00000    0.00000    0.00000    0.00000   -0.00000 
Auto vetores:
   0.29270   -0.92358    0.19477   -0.00209   -0.15293 
   0.11908   -0.09104   -0.85454   -0.39258   -0.30523 
   0.91098    0.34417    0.17366   -0.09404   -0.11249 
   0.14540   -0.00580   -0.34839    0.91489   -0.14289 
   0.22162   -0.14221   -0.28335    0.00000    0.92216 


In [17]:
n = len(auto_val)
vec_auto = []
for j in range(n):
    val = auto_val[j][j]
    vec = []
    for i in range(n):
        vec.append(auto_vec[i][j])
    vec_auto.append((val, vec))
vec_auto.sort(reverse=True)

V = [[0 for j in range(n)] for i in range(n)]
for i in range(n):
    for j in range(n):
        V[j][i] = vec_auto[i][1][j]

print("Matriz V:")
printMatrix(V)

Matriz V:
   0.29270   -0.92358    0.19477   -0.00209   -0.15293 
   0.11908   -0.09104   -0.85454   -0.39258   -0.30523 
   0.91098    0.34417    0.17366   -0.09404   -0.11249 
   0.14540   -0.00580   -0.34839    0.91489   -0.14289 
   0.22162   -0.14221   -0.28335    0.00000    0.92216 


# Achando a Matriz "E"

In [18]:
AAT = matrixByMatrix(A,Transposta(A))
auto_val, auto_vec = metodoQR(AU, tolerancia)
lamb = []
n = len(auto_val)
for i in range(n):
    lamb.append(auto_val[i][i])
lamb.sort(reverse=True)
print("Autovalores =",lamb)

Autovalores = [4290.807677050978, 1668.454365915156, 479.73795703386537]


In [19]:
n = len(A)
m = len(A[0])
E = [[0 for i in range(m)] for j in range(n)]
for i in range(n):
    E[i][i] = sqrt(lamb[i])
print("Matriz E:")
printMatrix(E)

Matriz E:
  65.50426    0.00000    0.00000    0.00000    0.00000 
   0.00000   40.84672    0.00000    0.00000    0.00000 
   0.00000    0.00000   21.90292    0.00000    0.00000 


# Resposta

In [20]:
print("Matriz U:")
printMatrix(U)
print("Matriz E:")
printMatrix(E)
print("Matriz V:")
printMatrix(V)

Matriz U:
   0.31146   -0.93741    0.15576 
   0.15582   -0.11132   -0.98149 
   0.93740    0.32997    0.11140 
Matriz E:
  65.50426    0.00000    0.00000    0.00000    0.00000 
   0.00000   40.84672    0.00000    0.00000    0.00000 
   0.00000    0.00000   21.90292    0.00000    0.00000 
Matriz V:
   0.29270   -0.92358    0.19477   -0.00209   -0.15293 
   0.11908   -0.09104   -0.85454   -0.39258   -0.30523 
   0.91098    0.34417    0.17366   -0.09404   -0.11249 
   0.14540   -0.00580   -0.34839    0.91489   -0.14289 
   0.22162   -0.14221   -0.28335    0.00000    0.92216 


In [21]:
print("Matrix A:")
printMatrix(A)
print("Matrix U.E.V^t:")
printMatrix(matrixByMatrix(matrixByMatrix(U,E),Transposta(V)))

Matrix A:
  42.00000    3.00000    6.00000    2.00000    9.00000 
   3.00000   20.00000    4.00000    9.00000    9.00000 
   6.00000    4.00000   61.00000    8.00000   11.00000 
Matrix U.E.V^t:
  42.00000    3.00000    6.00000    2.00000    9.00000 
   3.00000   20.00000    4.00000    9.00000    9.00000 
   6.00000    4.00000   61.00000    8.00000   11.00000 


# Ajustando para A = UEV^t

In [22]:
for i in [1,-1]:
    for j in [1, -1]:
        for k in [1, -1]:
            Unew = [[j for j in i] for i in U] 
            Unew[0][0] = i*Unew[0][0]
            Unew[1][0] = i*Unew[1][0]
            Unew[2][0] = i*Unew[2][0]
            Unew[0][1] = j*Unew[0][1]
            Unew[1][1] = j*Unew[1][1]
            Unew[2][1] = j*Unew[2][1]
            Unew[0][2] = k*Unew[0][2]
            Unew[1][2] = k*Unew[1][2]
            Unew[2][2] = k*Unew[2][2]
            
            print(f"Case {i} {j} {k}")
            print("A:")
            printMatrix(A)
            print("UAV^t:")
            printMatrix(matrixByMatrix(matrixByMatrix(Unew,E),Transposta(V)))
            print()

Case 1 1 1
A:
  42.00000    3.00000    6.00000    2.00000    9.00000 
   3.00000   20.00000    4.00000    9.00000    9.00000 
   6.00000    4.00000   61.00000    8.00000   11.00000 
UAV^t:
  42.00000    3.00000    6.00000    2.00000    9.00000 
   3.00000   20.00000    4.00000    9.00000    9.00000 
   6.00000    4.00000   61.00000    8.00000   11.00000 

Case 1 1 -1
A:
  42.00000    3.00000    6.00000    2.00000    9.00000 
   3.00000   20.00000    4.00000    9.00000    9.00000 
   6.00000    4.00000   61.00000    8.00000   11.00000 
UAV^t:
  40.67101    8.83089    4.81503    4.37720   10.93342 
  11.37411  -16.74118   11.46664   -5.97905   -3.18270 
   5.04958    8.16996   60.15257    9.70006   12.38268 

Case 1 -1 1
A:
  42.00000    3.00000    6.00000    2.00000    9.00000 
   3.00000   20.00000    4.00000    9.00000    9.00000 
   6.00000    4.00000   61.00000    8.00000   11.00000 
UAV^t:
 -28.72769   -3.97189   32.35660    1.55578   -1.89057 
  -5.39901   19.17208    7.12988    8